In [1]:
import numpy as np
import random as rn
import findspark
import sys
import time
import numba
from numba import cuda, float32, float64
findspark.init("/opt/spark")

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.accumulators import AccumulatorParam
sc = SparkContext("local","PSO Distribuido")
spark = SparkSession(sc)

In [3]:
class ListParam(AccumulatorParam):
    def zero(self, v):
        return []
    def addInPlace(self, variable, value):
        variable.append(value)
        return variable

In [4]:
def MSE(y, pred):
    n = len(y)
    if n != len(pred.value):
        print("error: datos y predicción de distintos tamaños")
        return -1

    resultado = 0.0

    for i in range(n):
        resultado += (y[i] - pred.value[i])**2

    resultado = resultado / n

    return resultado

In [5]:
V_max= 10.0
W= 1.0
c_1 = 0.8
c_2 = 0.2
objetivo_ =[50,50,50]
objetivo =sc.broadcast(objetivo_)
ind=0
filas_x=[0,0,0]

In [6]:
#Número dimensiones de los vectores
n=3
#Número de partículas
m= 10
#Número de iteraciones
I = 100
posiciones_=[]
mejor_posiciones_locales_=[]
velocidades_=[]
particulas = np.array([posiciones_,velocidades_,mejor_posiciones_locales_])
mejor_pos_global_=[]
mejor_pos_global_arr= np.array([0 for j in range(n)])
best_local_fitness_= []
best_local_fitness_arr=np.array(best_local_fitness_)
#maximum float
max = sys.float_info.max
best_global_fitness = max
accum=sc.accumulator(0)
#accum=sc.accumulator(0.0)
#mejor_pos_global= sc.broadcast(mejor_pos_global_arr)
mejor_pos_global= mejor_pos_global_arr
best_local_fitness= sc.broadcast(best_local_fitness_arr)

In [7]:
def InitParticles(part,N,M,bgf,blf):
    #global best_global_fitness
    parts_ =[]
    for j in range(M):
        posicion=[rn.uniform(-100,100) for j in range(N)]
        velocidad=[rn.uniform(-100,100) for j in range(N)]
        fit = MSE(posicion,objetivo)
        #posición, velocidad, mejor posición local, mejor fitness local
        part_ = posicion + velocidad + posicion + [fit]
        parts_.append(part_)

        best_local_fitness_.append(fit)
        blf= np.array(best_local_fitness_)
        #print("bgf-> ",bgf)
        if fit < bgf:
            bgf = fit
            accum= fit
            mejor_pos_global = posicion
    parts = np.array(parts_)
    return blf,bgf,mejor_pos_global,parts 

In [8]:
def fitnessEval(part,N):
    #global mejor_pos_global
    #global best_global_fitness
    #global best_local_fitness
    best_fit_local = part[3*N:]
    filas=part[0:N]
    fit = MSE(filas,objetivo)
    if fit < best_fit_local:
        part[3*N:] = fit
        #mpl[ind] = part[0:1,ind]
        for k in range(N):
            part[2*N+k]=filas[k]
            
        #if fit < best_global_fitness:
            #accum += (fit-best_global_fitness)
            #best_global_fitness = fit
            #mejor_pos_global := filas
            #accum := fit
    #if ind<10:
        #ind+=1
    #return part,mpg,blf,bgf
    return part 

In [9]:
def modifyAccum(part,N):
    global local_accum
        
    local_accum.add([part[2*N:3*N],part[3*N:]])

In [10]:
def posEval(part,mpg,N):
#def posEval_x(part,N):
    global ind
    velocidades=part[N:2*N]
    mpl=part[2*N:]
    r_1 = rn.random()
    r_2 = rn.random()
    for k in range(N):
        velocidades[k] = W*velocidades[k] + c_1*r_1*(mpl[k] - part[k]) + c_2*r_2*(mpg[k] - part[k])
        if velocidades[k] > V_max:
            velocidades[k] = V_max
        elif velocidades[k] < -V_max:
            velocidades[k] = -V_max  
        part[k] = part[k] + velocidades[k]
        part[N+k] = velocidades[k]
    #return part,velocidades
    return part

In [11]:
best_local_fitness,best_global_fitness,mejor_pos_global,particulas=InitParticles(particulas,n,m,best_global_fitness,best_local_fitness)

In [12]:
rdd_master= sc.parallelize(particulas)
tiempo_fitness = 0.0
tiempo_poseval = 0.0
tiempo_global = 0.0
tiempo_collect = 0.0
tiempo_foreach = 0.0

#print("rdd_master antes:", rdd_master.collect())
#print("antes:",mejor_pos_global)

start = time.time()

for i in range(I):
    local_accum = sc.accumulator([], ListParam()) 
    start_fitness = time.time()
    rdd_fitness = rdd_master.map(lambda x:fitnessEval(x,n))
    end_fitness = time.time()
    
    tiempo_fitness += end_fitness - start_fitness
    
    start_foreach = time.time()
    rdd_fitness.foreach(lambda x: modifyAccum(x,n))
    #rdd_eval = sc.broadcast(rdd_fitness.collect())
    end_foreach = time.time()
    
    tiempo_foreach += end_foreach - start_foreach
    
    #print("local_accum: ", local_accum.value)
    
    
    start_global = time.time()
    #for j in range(m):
        #blf = rdd_eval.value[j][3*n:]
        #if blf < best_global_fitness:
            #best_global_fitness = blf
            #mejor_pos_global = rdd_eval.value[j][2*n:3*n]
            
    
    for j in range(m):
        #print("local_accum[",j,"]: ", local_accum.value[0][j][1])
        blf = local_accum.value[0][j][1]
        if blf < best_global_fitness:
            best_global_fitness = blf
            mejor_pos_global = local_accum.value[0][j][0]
    end_global = time.time()
    
    tiempo_global += end_global - start_global
    
    
    start_poseval = time.time()
    resultado = rdd_fitness.map(lambda x:posEval(x,mejor_pos_global,n))
    end_poseval = time.time()
    
    tiempo_poseval += end_poseval - start_poseval
    
    start_collect = time.time()
    resultado = resultado.collect()
    end_collect = time.time()
    
    tiempo_collect += end_collect - start_collect
    
    #for j in range(m):
    #    particulas[j] = resultado[j][:]
        
    rdd_master= sc.parallelize(resultado)
    
end = time.time()  

#print("despues:",mejor_pos_global)
#print("rdd_master despues:", rdd_master.collect())

tiempo = end - start
resultado = rdd_master.collect()
print("Tiempo de ejecucion(s): ", tiempo)
print("Tiempo de ejecucion fitness(s): ", tiempo_fitness)
print("Tiempo de ejecucion poseval(s): ", tiempo_poseval)
print("Tiempo de ejecucion global fitness(s): ", tiempo_global)
print("Tiempo de ejecucion collect(s): ", tiempo_collect)
print("Tiempo de ejecucion foreach(s): ", tiempo_foreach)
print("mejor_pos_global-> ",mejor_pos_global)
print("mejor fitness global-> ", best_global_fitness, MSE(mejor_pos_global,objetivo))
#print("partículas: ",particulas)
print("local_accum: ", local_accum.value)
print("accum:", accum.value)

Tiempo de ejecucion(s):  21.162997007369995
Tiempo de ejecucion fitness(s):  0.04630422592163086
Tiempo de ejecucion poseval(s):  0.005309343338012695
Tiempo de ejecucion global fitness(s):  0.004915714263916016
Tiempo de ejecucion collect(s):  9.748331546783447
Tiempo de ejecucion foreach(s):  10.986859798431396
mejor_pos_global->  [49.92161862 50.10112005 50.37591518]
mejor fitness global->  [0.05256038] 0.0525603761550695
local_accum:  [[[array([49.51993463, 49.39639754, 49.59246237]), array([0.25362854])], [array([50.72318609, 49.78919517, 51.31475908]), array([0.76534275])], [array([49.99228139, 50.65707204, 50.85582228]), array([0.38807834])], [array([50.71483441, 49.4214018 , 49.94524738]), array([0.28292065])], [array([50.45346818, 48.62794578, 48.87827958]), array([1.11547429])], [array([49.79708202, 50.35384037, 49.88492849]), array([0.05987339])], [array([50.93569125, 50.305694  , 50.75370199]), array([0.51234454])], [array([49.92161862, 50.10112005, 50.37591518]), array([0.